In [ ]:
import numpy as np
import pandas as pd

from flagger import BayesFlaggerBeta, SVMFlagger
from flagger_tools import BayesFlaggerBetaTest, SVMFlaggerTest, ModelError
from flagger_tools import get_sampler_data
from matplotlib import pyplot as plt
from tmvbeta import TMVBeta

### Real World Data

Create sampler from data

In [ ]:
r = 0.2
P0 = pd.read_csv("Data/sample_reference_group.csv")
P1 = pd.read_csv("Data/sample_critical_group.csv")
sampler = get_sampler_data(r, P0, P1)

Initialize Bayes and SVM flagger

In [ ]:
K = 40
M = 3
bayes_flagger_pol_1 = BayesFlaggerBeta(K, M, rule="detection")
bayes_flagger_pol_3 = BayesFlaggerBeta(K, M, rule="mixed")
svm_flagger = SVMFlagger(K, M)

Initialize testing classes

In [ ]:
N = 100
bayes_test_pol_1 = BayesFlaggerBetaTest(bayes_flagger_pol_1, sampler, N)
bayes_test_pol_3 = BayesFlaggerBetaTest(bayes_flagger_pol_3, sampler, N)
svm_test = SVMFlaggerTest(svm_flagger, sampler, N)

Simulate flagging under given model

In [ ]:
T = 100
total_pol_1, detected_pol_1, model_pol_1, _ = bayes_test_pol_1.run(T)
total_pol_3, detected_pol_3, model_pol_3, phi = bayes_test_pol_3.run(T)
total_svm, detected_svm = svm_test.run(T)

Plot detection rates

In [ ]:
plt.plot(range(1, T+1), detected_pol_1[1:]/total_pol_1[1:], label="Detection-Greedy")
plt.plot(range(1, T+1), detected_pol_3[1:]/total_pol_3[1:], label="Mixed")
plt.plot(range(1, T+1), detected_svm[1:]/total_svm[1:], label="SVM")
plt.title("Detection Rate")
plt.ylabel("Detection Rate")
plt.xlabel("Administration")
plt.legend()
plt.grid()

Plot posterior means of $R$

In [ ]:
r_est_pol_1 = np.array([model.v[1] / model.v.sum() for model in model_pol_1])
r_est_pol_3 = np.array([model.v[1] / model.v.sum() for model in model_pol_3])

In [ ]:
plt.plot(range(T+1), r_est_pol_1, label="Detection-Greedy")
plt.plot(range(T+1), r_est_pol_3, label="Mixed")
plt.title(r"Posterior Mean of $R$")
plt.ylabel("Posterior Mean")
plt.xlabel("Administration")
plt.legend()
plt.grid()